In [1]:
import os

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class MaskedSelfAttention(nn.Module):

    def __init__(self, d_model=2, row_dim=0, col_dim=0):
        super().__init__()
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = row_dim
        self.col_dim = col_dim


    def forward(self, token_encodings, mask=None):

        q = self.W_q(token_encodings)
        k = self.W_k(token_encodings)
        v = self.W_v(token_encodings)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))
        scaled_sims = sims / torch.tensor(k.size(self.col_dim) ** 0.5)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
        
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attention_percents, v)
        return attention_scores

In [5]:
encodings_matrix = torch.tensor([
    [1.16, 0.23],
    [0.57, 1.36],
    [4.41, -2.16]
])

torch.manual_seed(42)

masked_self_attention = MaskedSelfAttention(d_model=2, row_dim=0, col_dim=1)

mask = torch.tril(torch.ones(3, 3))
mask = mask == 0

mask

tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

In [6]:
masked_self_attention(encodings_matrix, mask)

tensor([[ 0.6038,  0.7434],
        [-0.0062,  0.6072],
        [ 3.4989,  2.2427]], grad_fn=<MmBackward0>)

In [7]:
masked_self_attention.W_q.weight.transpose(0, 1), masked_self_attention.W_k.weight.transpose(0, 1), masked_self_attention.W_v.weight.transpose(0, 1)

(tensor([[ 0.5406, -0.1657],
         [ 0.5869,  0.6496]], grad_fn=<TransposeBackward0>),
 tensor([[-0.1549, -0.3443],
         [ 0.1427,  0.4153]], grad_fn=<TransposeBackward0>),
 tensor([[ 0.6233,  0.6146],
         [-0.5188,  0.1323]], grad_fn=<TransposeBackward0>))

In [8]:
masked_self_attention.W_q(encodings_matrix), masked_self_attention.W_k(encodings_matrix), masked_self_attention.W_v(encodings_matrix)

(tensor([[ 0.7621, -0.0428],
         [ 1.1063,  0.7890],
         [ 1.1164, -2.1336]], grad_fn=<MmBackward0>),
 tensor([[-0.1469, -0.3038],
         [ 0.1057,  0.3685],
         [-0.9914, -2.4152]], grad_fn=<MmBackward0>),
 tensor([[ 0.6038,  0.7434],
         [-0.3502,  0.5303],
         [ 3.8695,  2.4246]], grad_fn=<MmBackward0>))

In [9]:
q = masked_self_attention.W_q(encodings_matrix)
k = masked_self_attention.W_k(encodings_matrix)
v = masked_self_attention.W_v(encodings_matrix)

sims = torch.matmul(q, k.transpose(dim0=0, dim1=1))
sims

tensor([[-0.0990,  0.0648, -0.6523],
        [-0.4022,  0.4078, -3.0024],
        [ 0.4842, -0.6683,  4.0461]], grad_fn=<MmBackward0>)

In [10]:
scaled_sims = sims/(torch.tensor(2) **0.5)
scaled_sims

tensor([[-0.0700,  0.0458, -0.4612],
        [-0.2844,  0.2883, -2.1230],
        [ 0.3424, -0.4725,  2.8610]], grad_fn=<DivBackward0>)

In [11]:
masked_scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
masked_scaled_sims

tensor([[-6.9975e-02, -1.0000e+09, -1.0000e+09],
        [-2.8442e-01,  2.8833e-01, -1.0000e+09],
        [ 3.4241e-01, -4.7253e-01,  2.8610e+00]],
       grad_fn=<MaskedFillBackward0>)

In [12]:
attention_percents = F.softmax(masked_scaled_sims, dim=1)
attention_percents

tensor([[1.0000, 0.0000, 0.0000],
        [0.3606, 0.6394, 0.0000],
        [0.0722, 0.0320, 0.8959]], grad_fn=<SoftmaxBackward0>)

In [13]:
attention_scores = torch.matmul(attention_percents, v)
attention_scores

tensor([[ 0.6038,  0.7434],
        [-0.0062,  0.6072],
        [ 3.4989,  2.2427]], grad_fn=<MmBackward0>)